In [ ]:
#!pip install aquacrop==2.2
#!pip install statsmodels==0.13.2
!pip install dfply
!pip install seaborn
!pip install pyshp
!pip install geopandas

In [31]:
import statsmodels.api as sm
from statsmodels.tools.eval_measures import rmse
from aquacrop.utils import prepare_weather, get_filepath
from aquacrop import AquaCropModel, Soil, Crop, InitialWaterContent, IrrigationManagement
#from aquacrop.entities import IrrigationManagement
from os import chdir, getcwd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import datetime
import csv
import pickle 
#from dfply import *


import os 
import datetime
import shapefile as shp
import pandas as pd
import geopandas as gpd
from shapely import geometry
import math
from os import listdir
from os.path import isfile, join
import glob

In [ ]:
#wd=getcwd() # set working directory
#chdir(wd)

In [32]:
# GMD4 fields

#test = gpd.read_file('/vsicurl/https://github.com/Toblerity/Fiona/raw/master/tests/data/gre.shp')
# figure out how to read in download .shp files without downloading them
fields = gpd.read_file('~/eggs/GMD4/ModelExtent_GMD4model_BWilson_v1_20230111.shp')



# KS counties
# https://catalog.data.gov/dataset/tiger-line-shapefile-2019-state-kansas-current-county-subdivision-state-based
# site had wrong coordinates

# https://public.opendatasoft.com/explore/dataset/us-county-boundaries/export/?disjunctive.statefp&disjunctive.countyfp&disjunctive.name&disjunctive.namelsad&disjunctive.stusab&disjunctive.state_name&refine.state_name=Kansas

county_boundary = gpd.read_file('~/eggs/County Boundaries/USCountyBoundaries_WNdlovu_v1_20221220.shp')


# filter for countys in GMD-4
gmd4 = ['Cheyenne', 'Rawlins', 'Decatur', 'Sherman', 'Gove', 'Thomas', 'Sheridan', 'Graham', 'Wallace', 'Logan'] # list of gmd4 coundties
county_boundary = county_boundary[county_boundary['name'].isin(gmd4)] # filter for counties inside gmd4 list

# county_boundary and fields layers have same projection
fields = fields.to_crs({'init': 'epsg:4326'})
county_boundary = county_boundary.to_crs({'init': 'epsg:4326'})

# Execute spatial join
fields_county = county_boundary.sjoin(fields, how="inner", predicate='intersects') # has 93390 obs vs fields with 176407 obs
dup_fields = fields_county[fields_county['UID'].duplicated(keep= False)] # fields that appear in 2 counties

fields_county = fields_county.drop_duplicates(subset=['UID']) # dropped dupilcated observations based on UID
fields_county = fields_county[['UID', 'name', 'geometry']] # select useful variables


/home/jupyter-wndlovu/.local/lib/python3.9/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/home/jupyter-wndlovu/.local/lib/python3.9/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [29]:
test = gpd.read_file('https://drive.google.com/file/d/1XLhFovGtbRqqBRjPop1CXqD3gr4voRhU/view?usp=sharing')

DriverError: '/vsimem/9459ac8cb6c9424fb4430b1e5877b38a' not recognized as a supported file format.

In [6]:
g = pd.read_csv('https://drive.google.com/file/d/18k6wkiDBvN1CWifpGoklQeRPXxOztnYW/view?usp=sharing')

In [15]:
import pandas as pd

url = 'https://drive.google.com/file/d/18k6wkiDBvN1CWifpGoklQeRPXxOztnYW/view?usp=sharing'
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
df = pd.read_csv(path)

In [3]:
import pandas as pd
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

# Authenticate and create GoogleDrive instance
gauth = GoogleAuth()
drive = GoogleDrive(gauth)

# ID of the folder in Google Drive
folder_id = 'https://drive.google.com/drive/folders/1CqKIvHiJotyVmdh9QOFc7NrWhHb8MciR?usp=sharing'

# List files in the folder
file_list = drive.ListFile({'q': f"'{folder_id}' in parents and trashed=false"}).GetList()

# Read each CSV file from the folder
dfs = []
for file in file_list:
    if file['mimeType'] == 'text/csv':
        file_id = file['id']
        file_obj = drive.CreateFile({'id': file_id})
        csv_content = file_obj.GetContentString()
        df = pd.read_csv(pd.compat.StringIO(csv_content))
        dfs.append(df)

# Concatenate the DataFrames into a single DataFrame
combined_df = pd.concat(dfs)

# Now you can work with the combined DataFrame
# ...

# Print the combined DataFrame
print(combined_df)


InvalidConfigError: Invalid client secrets file Unknown client type: g_e.

In [25]:
df

,system:index,0,1,10,11,111,112,12,121,122,...,81,82,83,87,88,92,Year,masterid,status,.geo
0,1_0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,2020,0,0,"{""type"":""MultiPoint"",""coordinates"":[]}"
1,2_00000000000000001f2c,NaN,491.045999,NaN,NaN,NaN,NaN,NaN,4375.843846,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2020,365332,all,"{""type"":""MultiPoint"",""coordinates"":[]}"
2,2_00000000000000008dd9,NaN,491547.948860,NaN,NaN,NaN,NaN,NaN,5336.028727,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2020,362111,all,"{""type"":""MultiPoint"",""coordinates"":[]}"
3,2_00000000000000001b59,NaN,23879.317049,NaN,NaN,NaN,NaN,NaN,12352.128991,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2020,364342,all,"{""type"":""MultiPoint"",""coordinates"":[]}"
4,2_000000000000000028b6,NaN,7604.730978,NaN,NaN,NaN,NaN,NaN,6276.487332,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2020,388359,all,"{""type"":""MultiPoint"",""coordinates"":[]}"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176403,2_000300000000000060dc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,261.551746,29.969471,...,NaN,NaN,NaN,NaN,NaN,NaN,2020,555245,all,"{""type"":""MultiPoint"",""coordinates"":[]}"
176404,2_000300000000000060e2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2020,555254,all,"{""type"":""MultiPoint"",""coordinates"":[]}"
176405,2_0003000000000000844b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2020,313584,all,"{""type"":""MultiPoint"",""coordinates"":[]}"
176406,2_0003000000000000a059,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,513.491518,...,NaN,NaN,NaN,NaN,NaN,NaN,2020,1529774,all,"{""type"":""MultiPoint"",""coordinates"":[]}"


In [13]:
url = 'https://drive.google.com/drive/folders/1gC0sqxtJdw6ywoi0Z_YfbvARdc-8rwnw?usp=sharing'
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-1]


In [ ]:
!pip install dask
import dask.dataframe as dd
import glob

# Folder path where the .csv files are located
folder_path = '/home/jupyter-wndlovu/eggs/GMD4_LandUse'

# Get a list of .csv file paths in the folder
csv_files = glob.glob(folder_path)

print(csv_files)
# Read the .csv files into a Dask DataFrame
ddf = dd.read_csv(csv_files)

print(ddf)
# Perform operations on the Dask DataFrame
# ...

# Compute and display the result
result = ddf.compute()
print(result)


In [ ]:
# crop data data from 2006 - 2020
#crops_irrigation = pd.read_csv(wd + '/data/water/Kansas/IrrigationDepth_GMD4_WNdlovu_v1_20230123.csv')
#crops_field = pd.read_csv(wd + '/data/agricLand/landUse_landCover/FieldsAttributesAroundSD6KS_LandCover_AnnualCDL.csv')

# merge irrigation status files

# Folder path with .csv files 
landUse_path = '/home/jupyter-wndlovu/eggs/GMD4_LandUse'

# Get a list of .csv file paths in the folder
landUse_files = glob.glob(f"{landUse_path}/*.csv")

#print(landUse_files)
# read .csv file
landUse_dfs_list = []  # List to store the dataframes
for file in landUse_files:
    df = pd.read_csv(file) 
    landUse_dfs_list.append(df)

landUse_df = pd.concat(landUse_dfs_list, ignore_index=True) # merge dataframes in list 
landUse_df = landUse_df.drop(['system:index', 'status', '.geo'], axis=1) #drop system index variable

# replace all nan with 0 except for year and masterid cols
cols = ['Year', 'masterid']

landUse_df.loc[:, ~landUse_df.columns.isin(cols)] = landUse_df.loc[:, ~landUse_df.columns.isin(cols)].fillna(0)

# make year and masterid indexes
landUse_df = landUse_df.set_index(['Year', 'masterid'])
landUse_df['totalCover'] = landUse_df.sum(axis=1, numeric_only=True) #calculate (tatal land used) rowwise sum for all crops 

landUse_df = landUse_df[['1', '4', '5', '24', 'totalCover']] # select only the crops of interest "Corn", "Sorghum", "Soybeans", "Winter Wheat"]

# calculate % land cover
landUse_df['1'] = landUse_df['1']/landUse_df['totalCover']
landUse_df['4'] = landUse_df['4']/landUse_df['totalCover']
landUse_df['5'] = landUse_df['5']/landUse_df['totalCover']
landUse_df['24'] = landUse_df['24']/landUse_df['totalCover']

landUse_df = landUse_df.drop(['totalCover'], axis=1) # drop totalCover 
landUse_df = landUse_df.melt(ignore_index=False) #pivot longer on the pct covers
landUse_df = landUse_df.reset_index()
landUse_df = landUse_df.rename(columns={'masterid': 'UID', 'variable': 'CropCode', 'value': 'pctcov'}) # rename columns
landUse_df = landUse_df[landUse_df['UID'] != 0] # remove place holder row with uid of 0
landUse_df['CropCode'] = landUse_df['CropCode'].astype(int)
#x = landUse_df.head(5)

In [ ]:
# county level yield for different crops (corn, wheat, sorghum and soybeans)
corn_gmd4 = pd.read_csv(wd + '/data/agricLand/yield/Kansas/corn.csv')
corn_gmd4['Irrig_status'] = np.where(corn_gmd4['Data Item'].str.contains("NON-IRRIGATED"), 'rainfed', 'irrigated')
corn_gmd4 = corn_gmd4[["Year", "State", "County", "Commodity", "Data Item", "Irrig_status", "Value"]]

wheat_gmd4 = pd.read_csv(wd + '/data/agricLand/yield/Kansas/winter_wheat.csv')
wheat_gmd4['Irrig_status'] = np.where(wheat_gmd4['Data Item'].str.contains("NON-IRRIGATED"), 'rainfed', 'irrigated')
wheat_gmd4 = wheat_gmd4[["Year", "State", "County", "Commodity", "Data Item", "Irrig_status", "Value"]]

soybeans_gmd4 = pd.read_csv(wd + '/data/agricLand/yield/Kansas/soybeans.csv')
soybeans_gmd4['Irrig_status'] = np.where(soybeans_gmd4['Data Item'].str.contains("NON-IRRIGATED"), 'rainfed', 'irrigated')
soybeans_gmd4 = soybeans_gmd4[["Year", "State", "County", "Commodity", "Data Item", "Irrig_status", "Value"]]

sorghum_gmd4 = pd.read_csv(wd + '/data/agricLand/yield/Kansas/sorghum.csv')
sorghum_gmd4['Irrig_status'] = np.where(sorghum_gmd4['Data Item'].str.contains("NON-IRRIGATED"), 'rainfed', 'irrigated')
sorghum_gmd4 = sorghum_gmd4[["Year", "State", "County", "Commodity", "Data Item", "Irrig_status", "Value"]]

In [ ]:
# irrigation status
irrig_status = pd.read_csv(wd + '/data/agricLand/irrigationStatus/Kansas/AIM_HPA_GMD4_2000_2020_Deines_etal_RSE_v01_samsPolygons.csv')
irrig_status['0'] = irrig_status['0'].fillna(0) # replace nan with 0
irrig_status['1'] = irrig_status['1'].fillna(0) # replace nan with 0
irrig_status['IrrigatedPrc'] = 1 - (irrig_status['0'] / (irrig_status['0'] + irrig_status['1']))
irrig_status = irrig_status.assign(irrig_management = np.where(irrig_status['IrrigatedPrc'] <= 0.5, 'rainfed', 'irrigated')) # variable with irrigation status
irrig_status = irrig_status[['UID', 'year', 'IrrigatedPrc', 'irrig_management']]
irrig_status = irrig_status.rename(columns={'year': 'Year'})
#y = irrig_status.tail(5)

In [ ]:
# Create lists of crop codes, crop names, and crop groups to match them up in a single dataframe
CropCode = [1, 4, 5, 24]

crop_name = ["Corn", 
             "Sorghum", 
             "Soybeans",  
             "Winter Wheat"]


crops = pd.DataFrame(zip(CropCode, crop_name), columns=['CropCode', 'cropName'])

# new column with crop name int he crops_fields (full df)
crops_field = landUse_df.merge(crops, on='CropCode', how='left')

# one df with field and crop data
crops_irrig = crops_field.merge(irrig_status, on=['UID', 'Year'], how='inner') # create a dataframe which shows the field UID, irrigation status and the crop grown 
crops_irrig = crops_irrig.merge(fields_county, on=['UID'], how='inner') # merge to get df with the county


# make dataframe with county+crop+irrig_mngt polygon
from shapely.ops import unary_union

#mergedPolys = unary_union(polys)

crops_irrig_gdf = gpd.GeoDataFrame(crops_irrig) # transform to geodataframe
crops_irrig_gdf2 = crops_irrig_gdf.groupby(['name', 'cropName', 'irrig_management']).agg({'geometry': unary_union}) # combine polygons